In [1]:
# 1. 必要なライブラリをインストール
!pip install pandas requests tqdm lxml


[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [1]:
# 2. メインのPythonコード
import pandas as pd
import requests
import time
from tqdm import tqdm
import io
import re # 正規表現ライブラリをインポート

def get_large_cap_stocks_fast(market_cap_threshold_yen: int = 3000_0000_0000):
    """
    【最終回答・完全版】提供されたCSVデータの構造に基づき、正規表現でデータを抽出する。
    """
    print("Yahoo!ファイナンスの時価総額ランキングから高速にデータを取得します...")
    start_time = time.time()
    
    # --- スクレイピング部分 (ここは既に正常動作を確認済み) ---
    base_url = "https://finance.yahoo.co.jp/stocks/ranking/marketCapitalHigh?market=all&page={}"
    all_dfs = []
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'}
    
    max_page = 10 
    for page in tqdm(range(1, max_page + 1), desc="ランキングページ取得中"):
        try:
            url = base_url.format(page)
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()
            tables = pd.read_html(io.StringIO(response.text))
            
            if len(tables) > 0 and '時価総額(百万円)' in tables[0].columns:
                df = tables[0]
                all_dfs.append(df)
                temp_market_cap = pd.to_numeric(df['時価総額(百万円)'].astype(str).str.replace(',', '').str.replace('百万円',''), errors='coerce')
                if temp_market_cap.min() < (market_cap_threshold_yen / 1_000_000):
                    print(f"\nページ {page} で閾値以下の企業を検出したため、取得を終了します。")
                    break
            else:
                break
            time.sleep(0.5)
        except Exception:
            break

    if not all_dfs:
        print("データを取得できませんでした。")
        return None

    # --- データ整形処理（提供されたデータ構造に基づき全面改修） ---
    full_df = pd.concat(all_dfs, ignore_index=True)
    
    # ★★★ ここが最終解決策 ★★★
    
    # 1. 正規表現で「銘柄コード」を抽出する (例: 7203, 130A)
    code_pattern = r'(\d{4}[A-Z]?)'
    full_df['銘柄コード'] = full_df['名称・コード・市場'].str.extract(code_pattern)

    # 2. 正規表現で「銘柄名」を抽出する (銘柄コードの直前まで)
    name_pattern = r'(.+?)\d{4}[A-Z]?'
    full_df['銘柄名'] = full_df['名称・コード・市場'].str.extract(name_pattern)
    # (株)などの不要な文字を削除
    if '銘柄名' in full_df.columns:
        full_df['銘柄名'] = full_df['銘柄名'].str.replace('(株)', '', regex=False)

    # 3. 「時価総額(百万円)」から不要な文字とカンマを削除し、数値に変換
    market_cap_str = full_df['時価総額(百万円)'].astype(str)
    market_cap_cleaned = market_cap_str.str.replace(',', '').str.replace('百万円', '')
    full_df['時価総額_百万円'] = pd.to_numeric(market_cap_cleaned, errors='coerce')

    # --- フィルタリングと最終整形 ---
    # 必要な情報が正しく抽出できた行のみを対象とする
    full_df.dropna(subset=['銘柄コード', '銘柄名', '時価総額_百万円'], inplace=True)
    
    market_cap_threshold_million = market_cap_threshold_yen / 1_000_000
    large_cap_df = full_df[full_df['時価総額_百万円'] >= market_cap_threshold_million].copy()
    
    large_cap_df['時価総額_億円'] = (large_cap_df['時価総額_百万円'] / 100).round(0).astype(int)
    result_df = large_cap_df[['銘柄コード', '銘柄名', '時価総額_億円']].reset_index(drop=True)

    end_time = time.time()
    print(f"\n処理が完了しました。所要時間: {end_time - start_time:.2f} 秒")
    
    return result_df

if __name__ == '__main__':
    threshold = 3000_0000_0000
    result = get_large_cap_stocks_fast(market_cap_threshold_yen=threshold)

    if result is not None and not result.empty:
        print("\n--- 抽出結果：時価総額3000億円以上の企業リスト ---")
        pd.set_option('display.max_rows', None)
        print(result)
        print(f"\n合計: {len(result)} 社")
    else:
        print("\n条件に合う企業が見つかりませんでした。")

Yahoo!ファイナンスの時価総額ランキングから高速にデータを取得します...


ランキングページ取得中:  80%|████████  | 8/10 [00:07<00:01,  1.13it/s]


ページ 9 で閾値以下の企業を検出したため、取得を終了します。

処理が完了しました。所要時間: 7.13 秒

--- 抽出結果：時価総額3000億円以上の企業リスト ---
    銘柄コード                          銘柄名  時価総額_億円
0    7203                       トヨタ自動車   395270
1    8306            三菱ＵＦＪフィナンシャル・グループ   236225
2    6758                      ソニーグループ   221824
3    6501                        日立製作所   182804
4    7974                          任天堂   165323
5    9983                  ファーストリテイリング   146477
6    8316              三井住友フィナンシャルグループ   139063
7    9432                       日本電信電話   137908
8    6861                        キーエンス   136634
9    9984                   ソフトバンクグループ   131991
10   6098                リクルートホールディングス   121552
11   4519                         中外製薬   120707
12   8001                        伊藤忠商事   119295
13   7011                        三菱重工業   116391
14   8058                         三菱商事   113170
15   8035                     東京エレクトロン   113121
16   8766                 東京海上ホールディングス   111940
17   9434                       ソフトバンク   10390